In [1]:
import subprocess
subprocess.check_call(['pip3', 'install', 's3fs'])
subprocess.check_call(['pip3', 'install', 'boto3'])

0

In [2]:
import pandas as pd
import json
from lyft_analysis.db import hive, presto
import matplotlib.pyplot as plt
import boto3
import seaborn as sns

{"ts": "2022-05-31T18:46:32.072Z", "uuid": "f8d7722a-a54c-484c-94b3-94f11e040647", "name": "lyft_configset.configset", "lvlname": "ERROR", "lvlno": 40, "pid": 108, "call_site": "site-packages/lyft_configset/configset.py:77", "revision": "py38.77cc7", "msg": "Neither a configset at /srv/configset/envoyclient/current/ or runtime directory at /srv/runtime_data/current/envoyclient were found. It's likely that this is misconfigured."}


In [3]:
S3_CLIENT = boto3.client("s3")

def read_csv_from_s3(bucket, key):
    
    response = S3_CLIENT.get_object(Bucket=bucket, Key=key)
    status = response.get("ResponseMetadata", {}).get("HTTPStatusCode")
    if status == 200:
        return pd.read_csv(response.get("Body"))
    else:
        raise Exception(f'Unsuccessful S3 get_object response. Status - {status}')

In [4]:
# df = pd.read_csv('s3://lyft-intrepid-production-iad/allocation/weekly_ride_challenge/aggregate/2021-10-18/INCENTIVE_DROP/final_output/final_output.csv')
bucket = 'lyft-intrepid-production-iad'
key = 'allocation/weekly_ride_challenge/aggregate/2022-05-30/INCENTIVE_DROP/final_output/final_output.csv'

df = read_csv_from_s3(bucket, key)

In [5]:
print(df.shape)

(387472, 22)


In [6]:
def preprocessing(df):
    df['offer_details_json'] = df['offer_details'].apply(lambda x: json.loads(x))
    df['tier_1_req'] = df['offer_details_json'].apply(lambda x: x["tier_1_req"])
    df['tier_1_bonus_minor'] = df['offer_details_json'].apply(lambda x: x["tier_1_bonus_minor"])
    df['tier_2_req'] = df['offer_details_json'].apply(lambda x: x["tier_2_req"])
    df['tier_2_bonus_minor'] = df['offer_details_json'].apply(lambda x: x["tier_2_bonus_minor"])
    df['tier_3_req'] = df['offer_details_json'].apply(lambda x: x["tier_3_req"])
    df['tier_3_bonus_minor'] = df['offer_details_json'].apply(lambda x: x["tier_3_bonus_minor"])
    
    df['tier_1_richness'] = df['tier_1_bonus_minor']/df['tier_1_req'] 
    df['tier_2_richness'] = df['tier_2_bonus_minor']/df['tier_2_req'] 
    df['tier_3_richness'] = df['tier_3_bonus_minor']/df['tier_3_req'] 
    
    return df

In [9]:
df = preprocessing(df)

In [10]:
df[df.tier_1_req >0].tier_1_req.describe()

count    217101.000000
mean         39.932925
std          30.212351
min          10.000000
25%          15.000000
50%          31.000000
75%          56.000000
max         322.000000
Name: tier_1_req, dtype: float64

In [11]:
max_req_by_region = df.groupby('region')[['tier_1_req','tier_2_req','tier_3_req']].max()
max_req_by_region.sort_values(by = "tier_1_req", ascending=False)

,tier_1_req,tier_2_req,tier_3_req
region,,,
LAX,322,64,97
BNA,253,51,76
MSP,202,40,61
ORF,199,40,60
BOS,198,40,59
...,...,...,...
EAZ,32,6,10
MOP,30,6,9
IPL,30,6,9


In [14]:
df.sort_values(by = "tier_1_req", ascending=False).head(30)

,user_lyft_id,treatment,experiment_name,experiment_variant,exposure_options,allocation_algo_name,allocation_algo_version,buckets,propensity,region,...,offer_details_json,tier_1_req,tier_1_bonus_minor,tier_2_req,tier_2_bonus_minor,tier_3_req,tier_3_bonus_minor,tier_1_richness,tier_2_richness,tier_3_richness
190380,1125031618165643812,wrc_req1_1.0_bonus1_0.1_req2_0.2_bonus2_2.0_re...,INTREPID_WEEKLY_RIDE_CHALLENGEweekly_ride_chal...,"Explore, Embeddings","both, both",EpsilonGreedy,1.0.0,small_treatments_exclusion_1,0.010000,LAX,...,"{'tier_1_req': 322, 'tier_1_bonus_minor': 4290...",322,42900,64,17100,97,38800,133.229814,267.187500,400.000000
62288,885531132229683720,wrc_req1_1.0_bonus1_0.1_req2_0.2_bonus2_2.0_re...,INTREPID_WEEKLY_RIDE_CHALLENGEweekly_ride_chal...,"Explore, Embeddings","both, both",EpsilonGreedy,1.0.0,small_treatments_exclusion_1,0.010000,LAX,...,"{'tier_1_req': 282, 'tier_1_bonus_minor': 3760...",282,37600,56,14900,85,34000,133.333333,266.071429,400.000000
283539,1307383651012950140,wrc_req1_1.0_bonus1_0.2_req2_0.2_bonus2_2.0_re...,INTREPID_WEEKLY_RIDE_CHALLENGEweekly_ride_chal...,"Targeted, Production","both, both",EpsilonGreedy,1.0.0,small_treatments_exclusion_1,1.000000,BNA,...,"{'tier_1_req': 253, 'tier_1_bonus_minor': 5380...",253,53800,51,21700,76,48500,212.648221,425.490196,638.157895
266505,1262645159384969358,wrc_req1_1.0_bonus1_0.1_req2_0.2_bonus2_2.0_re...,INTREPID_WEEKLY_RIDE_CHALLENGEweekly_ride_chal...,"Explore, Production","both, both",EpsilonGreedy,1.0.0,small_treatments_exclusion_1,0.016667,BNA,...,"{'tier_1_req': 248, 'tier_1_bonus_minor': 2640...",248,26400,50,10600,74,23600,106.451613,212.000000,318.918919
305905,1371260130002426670,wrc_req1_1.0_bonus1_0.2_req2_0.2_bonus2_2.0_re...,INTREPID_WEEKLY_RIDE_CHALLENGEweekly_ride_chal...,"Targeted, Production","both, both",EpsilonGreedy,1.0.0,small_treatments_exclusion_1,1.000000,LAX,...,"{'tier_1_req': 222, 'tier_1_bonus_minor': 5920...",222,59200,44,23400,67,53600,266.666667,531.818182,800.000000
22542,739933245043795970,wrc_req1_1.0_bonus1_0.2_req2_0.2_bonus2_2.0_re...,INTREPID_WEEKLY_RIDE_CHALLENGEweekly_ride_chal...,"Explore, Production","both, both",EpsilonGreedy,1.0.0,small_treatments_exclusion_1,0.010000,LAX,...,"{'tier_1_req': 206, 'tier_1_bonus_minor': 5490...",206,54900,41,21800,62,49600,266.504854,531.707317,800.000000
262399,1252429759464503464,wrc_req1_1.0_bonus1_0.2_req2_0.2_bonus2_2.0_re...,INTREPID_WEEKLY_RIDE_CHALLENGEweekly_ride_chal...,"Targeted, Production","both, both",EpsilonGreedy,1.0.0,small_treatments_exclusion_1,1.000000,MSP,...,"{'tier_1_req': 202, 'tier_1_bonus_minor': 4730...",202,47300,40,18700,61,42900,234.158416,467.500000,703.278689
186791,1119156478766450472,wrc_req1_1.0_bonus1_0.2_req2_0.2_bonus2_2.0_re...,INTREPID_WEEKLY_RIDE_CHALLENGEweekly_ride_chal...,"Targeted, Production","both, both",EpsilonGreedy,1.0.0,small_treatments_exclusion_1,1.000000,ORF,...,"{'tier_1_req': 199, 'tier_1_bonus_minor': 3410...",199,34100,40,13700,60,30900,171.356784,342.500000,515.000000
128373,1020727226560315022,wrc_req1_1.0_bonus1_0.1_req2_0.2_bonus2_2.0_re...,INTREPID_WEEKLY_RIDE_CHALLENGEweekly_ride_chal...,"Targeted, Production","both, both",EpsilonGreedy,1.0.0,small_treatments_exclusion_1,1.000000,BOS,...,"{'tier_1_req': 198, 'tier_1_bonus_minor': 2670...",198,26700,40,10800,59,23900,134.848485,270.000000,405.084746
24523,748363794323980290,wrc_req1_1.0_bonus1_0.2_req2_0.2_bonus2_2.0_re...,INTREPID_WEEKLY_RIDE_CHALLENGEweekly_ride_chal...,"Explore, Embeddings","both, both",EpsilonGreedy,1.0.0,small_treatments_exclusion_1,0.013333,SAN,...,"{'tier_1_req': 194, 'tier_1_bonus_minor': 4840...",194,48400,39,19500,58,43400,249.484536,500.000000,748.275862


In [17]:
df.groupby('experiment_variant')[['experiment_variant']].count()

,experiment_variant
experiment_variant,
"Control, Embeddings",38523
"Control, Production",38898
"Explore, Embeddings",11119
"Explore, Production",11370
"Targeted, Embeddings",143887
"Targeted, Production",143675
